# IbPRIA 2025 - Data-Efficient Strategies for Object Detection

> 📌 **Note**: You can run this notebook on:
>
> - 💻 Your local machine (Python ≥ 3.8, see `requirements.txt`)
> - 🌐 [Google Colab](https://colab.research.google.com/github/dinisdcosta/IbPRIA2015---Data-Efficient-Strategies-for-Object-Detection/blob/main/old.ipynb)
>
> Instructions are provided inline for both environments.


In [2]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("✅ Running on Google Colab")
    print("Installing dependencies...")
    !pip install -r requirements.txt > /dev/null 2>&1
    # e.g., clone repo or mount Google Drive
else:
    print("✅ Running locally")
    print("Installing dependencies...")
    !pip install -r requirements.txt > /dev/null 2>&1

✅ Running locally
Installing dependencies...


## Annotations